In [62]:
!pip install transformers

In [28]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests
import re

In [122]:
MODEL_ID = "facebook/bart-large-cnn" #@param {type: "string"}

#Kipróbált modellek:
# "Falconsai/text_summarization" -> Magyar szöveg kuka
# "SZTAKI-HLT/mT5-base-HunSum-1" -> Nem jól dolgozza fel a szöveget, értelmezhetetlen
# NYTK/summarization-hi-mt5-base-hungarian -> Tökéletes volt magyar szövegre, angolra nem fine tuningolt
# slauw87/bart_summarisation -> Fő problémánk vele az volt, hogy inkonzisztens valamely szövegekre tényleg emberinek tűnő választ adott mondjuk azt 10/4 esetben de a maradékban abszolút hülyeséget
# facebook/bart-large-cnn -> Legjobb opciónk volt mert a magyar szöveget is 10/8 esetben jól fordította de az angolt abszolút perfektül


In [123]:
summarizer = pipeline("summarization", model=MODEL_ID)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [124]:
URL = 'https://magyarugyved.blog.hu/' #@param {type:"string"}

In [125]:
r = requests.get(URL)

In [126]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [127]:
ARTICLE

'\n\n Répássy szándékosan mellébeszélt: szerinte a parlament felelőssége a választási rendszer alakítása, és ebbe a kormány nem kíván beleszólni A Fidesz nehezen viseli Budapest elvesztését, és mindent megtesz a főváros feletti ellenőrzés visszaszerzésére. A Mi Hazánk kezdeményezte: módosítsák az önkormányzati választási törvényt úgy, hogy a fővárosi képviselő-testület tagjait listán válasszák meg, amivel esetleg jól járhat a Fidesz és szélsőjobbos szatellitszervezete. De ha a demokratikus ellenzék összefog, akár öngólt is rúghatnak? A lényeg az, hogy történjék bármi, csak a Fidesz!  Szuverenitásvédelem, avagy itt az új AVH? Nem a „dollárbaloldal”, hanem a Fidesz árusítja ki Magyarországot! Végre lehullott a lepel, és kiderült, mi fán terem a szuverenitásvédelmi törvény. Nem lettünk azonban igazán okosabbak a fideszes Kocsis Máté által jegyzett előterjesztéstől, mert az egész egy jogi rémálom. Első pillantásra a javaslat értelmezhetetlen, alkalmazhatatlan, és teljesen felesleges. Amúgy

In [128]:
max_chunk = 250 #@param {type:"int"};

In [129]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [130]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [131]:
len(chunks)

3

In [132]:
MAX_LENGTH = 120 #@param {type:"int"};
MIN_LENGTH = 30 #@param {type:"int"};

In [133]:
res = summarizer(chunks, max_length=MAX_LENGTH, min_length=MIN_LENGTH, do_sample=False)

In [134]:
res[0]

{'summary_text': 'A Fidesz nehezen viseli Budapest elvesztését, és mindent megtesz a főváros feletti ellenőrzés visszaszerzésére. A Mi Hazánk kezdeményezte: módosítsák az önkormányzati választási törvényt úgy.'}

In [135]:
' '.join([summ['summary_text'] for summ in res])

'A Fidesz nehezen viseli Budapest elvesztését, és mindent megtesz a főváros feletti ellenőrzés visszaszerzésére. A Mi Hazánk kezdeményezte: módosítsák az önkormányzati választási törvényt úgy. Baleset a Nemzeti Színházban: az eredeti helyszín megváltoztatása, a díszlet elbontása a bizonyítási eljárás eredményességét veszélyezteti Visszaépítik. Vidnyánszky nem le-, hanem csak felmondhat – egy igekötő meghatároz A Fidesz Antikrisztusa Az Iványi Gábor szervezetei ellen folytatott hadjárat egyszerűen felháborító!  Ami persze érthető, ha az Orbán két gyermekét megkeresztelő lelkész korábban úgy nyilatkozott. A pedofil képek birtoklásáért el'

In [136]:
text = ' '.join([summ['summary_text'] for summ in res])

In [137]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)